In [1]:
!python --version

Python 3.6.9


In [2]:
#!pip install -q -U pip
#!pip install -q numpy
#!pip install -q pandas
!pip install -q ckiptagger[gdown]
!pip install -q tqdm
#!pip install -q tensorflow==1.14.0
!pip install -q ipywidgets

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd './drive/My Drive/NLP/day31'

/content/drive/My Drive/NLP/day31


In [5]:
from ckiptagger import data_utils, construct_dictionary, WS, POS

In [6]:
# Downloads to ./data.zip (2GB) and extracts to ./data/
# data_utils.download_data_url("./") # iis-ckip
# data_utils.download_data_gdown("./") # gdrive-ckip

In [7]:
import pandas as pd
import numpy as np

#from ckiptagger import WS, POS
from tqdm.notebook import tqdm

In [8]:
df_train = pd.read_csv('news_clustering_train.tsv', sep='\t')
df_test = pd.read_csv('news_clustering_test.tsv', sep='\t')

In [9]:
df_train

,index,class,title
0,0,體育,亞洲杯奪冠賠率：日本、伊朗領銜 中國竟與泰國並列
1,1,體育,9輪4球本土射手僅次武磊 黃紫昌要搶最強U23頭銜
2,2,體育,如果今年勇士奪冠，下賽季詹姆斯何去何從？
3,3,體育,超級替補！科斯塔本賽季替補出場貢獻7次助攻
4,4,體育,騎士6天里發生了啥？從首輪搶七到次輪3-0猛龍
...,...,...,...
1795,1795,遊戲,LOL：麻辣香鍋韓服Rank不合成打野刀？電刀巨魔新套路連勝中
1796,1796,遊戲,穩住，我們能贏！因為我們擁有這種強大的力量
1797,1797,遊戲,騰訊是怎樣毀掉《鬥戰神》這款可能成神的作品的？
1798,1798,遊戲,LOL你不知道的黑科技打法！


In [10]:
train_titles = {row['index']: row['title'] for _, row in df_train.iterrows()}
train_classes = {row['index']: row['class'] for _, row in df_train.iterrows()}

test_titles = {row['index']: row['title'] for _, row in df_test.iterrows()}
test_classes = {row['index']: row['class'] for _, row in df_test.iterrows()}

In [11]:
all_news_class = ['體育', '財經', '科技', '旅遊', '農業', '遊戲']

In [12]:
train_titles.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

In [13]:
train_titles[0]

'亞洲杯奪冠賠率：日本、伊朗領銜 中國竟與泰國並列'

# 斷詞 + POS

In [14]:
ws = WS('./data/')
pos = POS('./data/')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:903: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


In [15]:
train_title_cuts = {}
for index, title in tqdm(train_titles.items()):
    ##########
    word_sentence_list = ws([title])
    #print(word_sentence_list)
    pos_sentence_list = pos(word_sentence_list)
    #print(pos_sentence_list)

    train_title_cuts[index] = list(zip(word_sentence_list[0], pos_sentence_list[0]))

In [16]:
test_title_cuts = {}
for index, title in tqdm(test_titles.items()):
    ##########
    word_sentence_list = ws([title])
    #print(word_sentence_list)
    pos_sentence_list = pos(word_sentence_list)
    #print(pos_sentence_list)
    #https://clay-atlas.com/blog/2019/09/24/python-chinese-tutorial-ckiptagger/
    test_title_cuts[index] = list(zip(word_sentence_list[0], pos_sentence_list[0]))

In [17]:
# 120是keys
train_title_cuts[120]

[('國腳', 'Na'),
 ('張呈棟', 'Nb'),
 ('：', 'COLONCATEGORY'),
 ('從', 'D'),
 ('沒', 'D'),
 ('想', 'VE'),
 ('過', 'Di'),
 ('自己', 'Nh'),
 ('會', 'D'),
 ('出', 'VC'),
 ('一', 'Neu'),
 ('本', 'Nf'),
 ('書', 'Na')]

# Bag of Words (BOW)

In [18]:
word2index = {}
index2word = {}
# 產生字與index對應的關係
word_set = set()
idx = 0
for train_title in train_title_cuts.values():
  for pairs in train_title:
    word_set.add(pairs[0])

for word in word_set:
  index2word[idx] = word
  word2index[word] = idx
  idx +=1


In [19]:
word2index['溫暖']

1940

In [21]:
index2word[1940]

'溫暖'

In [22]:
print(len(word2index.keys()))
print(len(index2word.keys()))

6690
6690


In [23]:
def get_bow_vector(pairs, word2index):
    vector = np.zeros(shape = (len(word2index)) )
    #####################
    for word , flag in pairs:
      idx = word2index[word]
      vector[idx] = 1
    return vector

In [24]:
get_bow_vector(train_title_cuts[120], word2index)

array([0., 0., 0., ..., 0., 0., 0.])

# 排除較無意義的詞性

In [25]:
pos_analysis = {}
for _, pairs in train_title_cuts.items():
    for word, flag in pairs:
        if flag not in pos_analysis:
            pos_analysis[flag] = set()
        pos_analysis[flag].add(word)

for flag, words in pos_analysis.items():
    print(flag, ':', list(words)[:100])
    print('=======================')

Nb : ['王者榮', '賞月夜', '朱嘯虎', '埃姆雷', '東皇', '石川佳純', '唐汪鎮', '埃爾克森', '湯米', '梵', '陳柯楓', '李白界', '穆謝奎', '斯威', '科爾', '安利', '盧克肖', '猛', '微眾', '微醫', '奧拉朱旺遠', '吉尼斯', '·龜茲', '洛瑞', '男乒', '璞麗', '老馬', '李大霄', '萬孚', '單薇恩', '安永', '浦發', '本澤馬', '維韋杜詹霍', '周雲傑', '火箭隊', '大神', '神帥', '特里', '劉國梁', '亞洲杯', '周琦', '塔克', '網易', '海富通', '美景美', '胤祥', '每日', '朱婷', '恆大', '龔方雄', 'NBA', '京新', '姆巴佩', '凱爾特', '章澤天', '馬克思', '朱之文', 'P60八核讓', '紫鑫', '馬克', '沃爾瑪', '小白', '昂科威', '高曉松', '·瞻', '劉偉', '蘇群', '安徒恩比盧克', '皮爾洛', '泰倫', '鎧單', '微博', '庫里', '后羿', 'J羅', '巴薩', '郎平', '溫氏', '騎士隊', '尤文', '梅西納', '馬里奧', '傑森', '亞亞·圖雷', '羅傑斯', '雷霆隊', '靜懿', '詹黑皮爾斯', '萊萬', '阿迪薩亞', '精准', '艾科麥佛鯊', '德比', '陰陽師', '卡帥', '綠軍', '斐濟', '白起', '張大仙']
VC : ['扔上', '超爭', '管理', '參考', '打野', '害', '帶動', '開', '撿到', '支持', '傳用', '啓定', '登記', '栽', '接受', '靠近', '上', '面試', '超越', '追', '種點', '看多', '攔', '邀請', '對', '保留', '造出', '丟棄', '虛擬', '絕殺', '焊', '募', '對付', '拍賣', '反擊', '炸', '扔丟', '學會', '製作', '分析', '收錄', '決殺', '開放', '監控', '摔出', '採取', '覲見', '花用', '指導', '看待', '追上', '詮釋', '引入', '織', '建', '觀看',

|         Type        |     Description    |
|:-------------------:|:------------------:|
| A                   | 非謂形容詞         |
| Caa                 | 對等連接詞         |
| Cab                 | 連接詞，如：等等   |
| Cba                 | 連接詞，如：的話   |
| Cbb                 | 關聯連接詞         |
| D                   | 副詞               |
| Da                  | 數量副詞           |
| Dfa                 | 動詞前程度副詞     |
| Dfb                 | 動詞後程度副詞     |
| Di                  | 時態標記           |
| Dk                  | 句副詞             |
| DM                  | 定量式             |
| I                   | 感嘆詞             |
| Na                  | 普通名詞           |
| Nb                  | 專有名詞           |
| Nc                  | 地方詞             |
| Ncd                 | 位置詞             |
| Nd                  | 時間詞             |
| Nep                 | 指代定詞           |
| Neqa                | 數量定詞           |
| Neqb                | 後置數量定詞       |
| Nes                 | 特指定詞           |
| Neu                 | 數詞定詞           |
| Nf                  | 量詞               |
| Ng                  | 後置詞             |
| Nh                  | 代名詞             |
| Nv                  | 名物化動詞         |
| P                   | 介詞               |
| T                   | 語助詞             |
| VA                  | 動作不及物動詞     |
| VAC                 | 動作使動動詞       |
| VB                  | 動作類及物動詞     |
| VC                  | 動作及物動詞       |
| VCL                 | 動作接地方賓語動詞 |
| VD                  | 雙賓動詞           |
| VF                  | 動作謂賓動詞       |
| VE                  | 動作句賓動詞       |
| VG                  | 分類動詞           |
| VH                  | 狀態不及物動詞     |
| VHC                 | 狀態使動動詞       |
| VI                  | 狀態類及物動詞     |
| VJ                  | 狀態及物動詞       |
| VK                  | 狀態句賓動詞       |
| VL                  | 狀態謂賓動詞       |
| V_2                 | 有                 |
|                     |                    |
| DE                  | 的之得地           |
| SHI                 | 是                 |
| FW                  | 外文               |
|                     |                    |
| COLONCATEGORY       | 冒號               |
| COMMACATEGORY       | 逗號               |
| DASHCATEGORY        | 破折號             |
| DOTCATEGORY         | 點號               |
| ETCCATEGORY         | 刪節號             |
| EXCLAMATIONCATEGORY | 驚嘆號             |
| PARENTHESISCATEGORY | 括號               |
| PAUSECATEGORY       | 頓號               |
| PERIODCATEGORY      | 句號               |
| QUESTIONCATEGORY    | 問號               |
| SEMICOLONCATEGORY   | 分號               |
| SPCHANGECATEGORY    | 雙直線             |
| WHITESPACE          | 空白               |

In [26]:
def get_bow_vector_with_selection(pairs, word2index):
    # 根據上列舉出來的文字以及詞性表，請列出想要排除的詞性
    excluded_flags = ['COLONCATEGORY','PAUSECATEGORY','WHITESPACE','COMMACATEGORY','QUESTIONCATEGORY','EXCLAMATIONCATEGORY',
               'DASHCATEGORY','PARENTHESISCATEGORY','PERIODCATEGORY','SEMICOLONCATEGORY','ETCCATEGORY','DOTCATEGORY',
               'I','T']
    vector = np.zeros(len(word2index))
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            vector[word2index[word]] += 1
    return vector

# Cosine Similarity

In [31]:
def cosine_similarity(bow1, bow2):
    ############
    numerator = np.dot(bow1,bow2)
    denominator = np.sqrt(np.dot(bow1,bow1))*np.sqrt(np.dot(bow2,bow2))
    similarity = numerator/denominator
    return similarity

In [32]:
bow1 = get_bow_vector(train_title_cuts[100], word2index)
bow2 = get_bow_vector(train_title_cuts[130], word2index)
cosine_similarity(bow1, bow2)

0.08703882797784893

In [33]:
train_title_cuts[100]

[('山東', 'Nc'),
 ('魯能', 'Nb'),
 ('有沒有', 'D'),
 ('可能', 'D'),
 ('拿到', 'VC'),
 ('今年', 'Nd'),
 ('的', 'DE'),
 ('中', 'A'),
 ('超', 'A'),
 ('冠軍', 'Na'),
 ('？', 'QUESTIONCATEGORY')]

In [34]:
train_title_cuts[130]

[('NBA', 'Nb'),
 ('和', 'Caa'),
 ('CBA', 'FW'),
 ('差距', 'Na'),
 ('在', 'P'),
 ('哪裡', 'Ncd'),
 ('？', 'QUESTIONCATEGORY'),
 ('6', 'Neu'),
 ('張', 'Nf'),
 ('圖', 'VF'),
 ('一目瞭然', 'VH'),
 ('！', 'EXCLAMATIONCATEGORY')]

# Group mean vector

In [35]:
group_vectors = {news_class: [] for news_class in all_news_class}
for index, pairs in sorted(train_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    news_class = train_classes[index]
    group_vectors[news_class].append(vector)

group_mean_vector = {}
for news_class, vectors in group_vectors.items():
    group_mean_vector[news_class] = np.mean(vectors, axis=0)
group_mean_vector

{'旅遊': array([0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.00333333]),
 '科技': array([0.        , 0.        , 0.00333333, ..., 0.        , 0.        ,
        0.        ]),
 '財經': array([0.        , 0.00333333, 0.        , ..., 0.        , 0.        ,
        0.        ]),
 '農業': array([0., 0., 0., ..., 0., 0., 0.]),
 '遊戲': array([0.00333333, 0.        , 0.        , ..., 0.00333333, 0.        ,
        0.        ]),
 '體育': array([0.        , 0.        , 0.        , ..., 0.        , 0.00333333,
        0.        ])}

# Group mean vector: 測試

In [36]:
classification = {news_class: [] for news_class in all_news_class}
for index, pairs in sorted(test_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    if np.sum(np.square(vector)) == 0:
        continue

    max_val = -2.0
    max_class = None
    for news_class, ref_vector in group_mean_vector.items():
        val = cosine_similarity(ref_vector, vector)
        if val > max_val:
            max_class = news_class
            max_val = val

    classification[max_class].append(index)

In [37]:
from collections import Counter

for group, ids in classification.items():
    counter = Counter([test_classes[id] for id in ids])
    print('predict', group, ': ', counter)

predict 體育 :  Counter({'體育': 65, '旅遊': 10, '遊戲': 8, '財經': 7, '農業': 7, '科技': 3})
predict 財經 :  Counter({'財經': 64, '科技': 21, '農業': 11, '體育': 9, '旅遊': 8, '遊戲': 5})
predict 科技 :  Counter({'科技': 57, '財經': 15, '農業': 12, '體育': 10, '旅遊': 7, '遊戲': 6})
predict 旅遊 :  Counter({'旅遊': 59, '財經': 10, '農業': 10, '體育': 5, '科技': 5, '遊戲': 5})
predict 農業 :  Counter({'農業': 57, '旅遊': 6, '科技': 4, '遊戲': 4, '體育': 2, '財經': 1})
predict 遊戲 :  Counter({'遊戲': 72, '科技': 9, '旅遊': 9, '體育': 8, '財經': 3, '農業': 3})
